In [1]:
import sys
v_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'
if v_path in sys.path:
    sys.path.remove(v_path)
import numpy as np
import pandas as pd
from keras.applications.resnet50 import ResNet50
from keras.optimizers import SGD, Adam
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense
from PIL import Image  
import os
import csv
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data.csv')

In [3]:
def make_square(fp):
    image = Image.open(fp)  
    image = image.resize((224, 224))
    r_img = np.array(image)
    if(r_img.shape == (224, 224)):
        return r_img.reshape(224, 224, 1)
    return r_img[:,:,0].reshape(224, 224, 1)

In [21]:
X = []
y = []
for i in range(data.shape[0]):
    print(i)
    fp1 = data['im1'][i]
    fp2 = data['im2'][i]
    if(os.path.exists(fp1) and os.path.exists(fp2)):
        image1 = make_square(fp1)
        image2 = make_square(fp2)
        datapoint = np.append(image1, image2, axis=2)
        X.append(datapoint)
        y.append(data['label'][i])
    else:
        print(fp1)
X = np.array(X)
y = np.array(y)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
np.save('resized_images', X)
np.save('resized_labels', y)

In [4]:
X = np.load('resized_images.npy')
y = np.load('resized_labels.npy')

In [5]:
X0 = np.zeros((X.shape[0], 224, 224)).reshape(X.shape[0], 224, 224, 1)

In [6]:
X0.shape

(678, 224, 224, 1)

In [7]:
X = np.append(X, X0, axis=3)

In [8]:
X.shape

(678, 224, 224, 3)

In [9]:
from keras.applications import InceptionResNetV2
conv_base = InceptionResNetV2(weights=None, include_top=False, input_shape=(224,224,3))
conv_base.load_weights('inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [10]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
conv_base.trainable = False

In [11]:
from keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 5, 5, 1536)        54336736  
_________________________________________________________________
flatten_1 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               9830656   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 64,167,649
Trainable params: 9,830,913
Non-trainable params: 54,336,736
_________________________________________________________________


In [ ]:
model.load_weights('Trained_InceptionNet')

In [14]:
model.fit(X, y, verbose=1, validation_split=0.33, batch_size=32, epochs=15, shuffle=True)

Train on 454 samples, validate on 224 samples
Epoch 1/15
454/454 [==============================] - 80s 177ms/step - loss: 0.1093 - acc: 0.9581 - val_loss: 0.0651 - val_acc: 0.9911
Epoch 2/15
454/454 [==============================] - 72s 158ms/step - loss: 0.0308 - acc: 0.9956 - val_loss: 0.0931 - val_acc: 0.9866
Epoch 3/15
454/454 [==============================] - 73s 162ms/step - loss: 0.0403 - acc: 0.9934 - val_loss: 0.4839 - val_acc: 0.7902
Epoch 4/15
454/454 [==============================] - 69s 152ms/step - loss: 0.0398 - acc: 0.9890 - val_loss: 0.0203 - val_acc: 0.9955
Epoch 5/15
454/454 [==============================] - 69s 152ms/step - loss: 0.0220 - acc: 0.9956 - val_loss: 0.0152 - val_acc: 0.9955
Epoch 6/15
454/454 [==============================] - 69s 152ms/step - loss: 0.0223 - acc: 0.9978 - val_loss: 0.1094 - val_acc: 0.9777
Epoch 7/15
454/454 [==============================] - 69s 151ms/step - loss: 0.0498 - acc: 0.9802 - val_loss: 0.1798 - val_acc: 0.9464
Epoch 8/1

In [47]:
model.save('Trained_InceptionNet')

In [22]:
# Testing
X_test = []
index = []
for i in range(301):
    print(i)
    fp1 = 'TestData(XraysOnly)/Test (' + str(i+1) + ')/AP/AP.jpg'
    fp2 = 'TestData(XraysOnly)/Test (' + str(i+1) + ')/LAT/LAT.jpg'
    if(os.path.exists(fp1) and os.path.exists(fp2)):
        img1 = make_square(fp1)
        img2 = make_square(fp2)
        datapoint = np.append(img1, img2, axis=2)
        X_test.append(datapoint)
        index.append(i+1)
X_test = np.array(X_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
X1 = np.zeros((X_test.shape[0], 224, 224)).reshape(X_test.shape[0], 224, 224, 1)

In [26]:
X_test = np.append(X_test, X1, axis=3)

In [27]:
X_test.shape

(299, 224, 224, 3)

In [29]:
y = model.predict(X_test)

In [31]:
for i in range(y.shape[0]):
    if(y[i] >= 0.5 ):
        y[i] = 1
    else:
        y[i] = 0

In [33]:
y = 1-y

In [44]:
X_test.shape

(299, 224, 224, 3)

In [45]:
with open('output1.csv', 'w', newline='') as file:
    fieldnames = ['Image ID', 'Label']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(X_test.shape[0]):
        writer.writerow({'Image ID': index[i], 'Label': y[i][0]})